# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), January 18, 2019**

In [1]:
using DataFrames # load package

## Обработка пропущенных значений

Тип `Missing` позволяет нам иметь дело с пропущенными значениями.

In [1]:
missing, typeof(missing)

(missing, Missing)

Массивы автоматически создают соответствующий тип объединения.

In [3]:
x = [1, 2, missing, 3]

4-element Array{Union{Missing, Int64},1}:
 1       
 2       
  missing
 3       

`ismissing` проверяет, отсутствует ли переданное значение

In [4]:
ismissing(1), ismissing(missing), ismissing(x), ismissing.(x)

(false, true, false, Bool[false, false, true, false])

Мы можем извлечь тип в сочетании с Missing из `Union` как `Missings.T`

(Это полезно для массивов!)

In [5]:
eltype(x), Missings.T(eltype(x))

(Union{Missing, Int64}, Int64)

`missing` сравнения производят `missing`.

In [6]:
missing == missing, missing != missing, missing < missing

(missing, missing, missing)

Это также верно, когда  `missing` сравниваются со значениями других типов.

In [7]:
1 == missing, 1 != missing, 1 < missing

(missing, missing, missing)

`isequal`, `isless`, и `===` возвращают тип `Bool`. Заметьте, что `missing` считается больше, чем любое числовое значение.

In [2]:
isequal(missing, missing), missing === missing, isequal(1, missing), isless(1, missing)

(true, true, false, true)

В следующих нескольких примерах мы видим, что многие (не все) функции обрабатывают `missing`.

In [9]:
map(x -> x(missing), [sin, cos, zero, sqrt]) # part 1

4-element Array{Missing,1}:
 missing
 missing
 missing
 missing

In [10]:
map(x -> x(missing, 1), [+, - , *, /, div]) # part 2 

5-element Array{Missing,1}:
 missing
 missing
 missing
 missing
 missing

In [11]:
using Statistics # нужно чтоб считать среднее
map(x -> x([1,2,missing]), [minimum, maximum, extrema, mean, float]) # part 3

5-element Array{Any,1}:
 missing                                   
 missing                                   
 (missing, missing)                        
 missing                                   
 Union{Missing, Float64}[1.0, 2.0, missing]

`skipmissing` возвращает итератор, перескакивая пропущенные значения. Мы можем использовать `collect` и` skipmissing` для создания массива, который исключает эти пропущенные значения.

In [12]:
collect(skipmissing([1, missing, 2, missing]))

2-element Array{Int64,1}:
 1
 2

Точно так же здесь мы объединяем `collect` и` Missings.replace` для создания массива, который заменяет все пропущенные значения некоторым значением (в данном случае `NaN`).

In [13]:
collect(Missings.replace([1.0, missing, 2.0, missing], NaN))

4-element Array{Float64,1}:
   1.0
 NaN  
   2.0
 NaN  

Еще один способ сделать это:

In [14]:
coalesce.([1.0, missing, 2.0, missing], NaN)

4-element Array{Float64,1}:
   1.0
 NaN  
   2.0
 NaN  

Вы можете использовать `recode`, если у вас есть однородные типы вывода.

In [15]:
recode([1.0, missing, 2.0, missing], missing=>NaN)

4-element Array{Float64,1}:
   1.0
 NaN  
   2.0
 NaN  

Вы можете использовать `unique` или `levels`, чтобы получить уникальные значения с или без пропусков, соответственно.

In [16]:
unique([1, missing, 2, missing]), levels([1, missing, 2, missing])

(Union{Missing, Int64}[1, missing, 2], [1, 2])

В следующем примере мы конвертируем `x` в `y` используя `allowmissing`, где `y` имеет тип допускающий missings.

In [17]:
x = [1,2,3]
y = allowmissing(x)

3-element Array{Union{Missing, Int64},1}:
 1
 2
 3

Затем мы конвертируем обратно с `disallowmissing`. Это не сработает, если `y` содержит пропущенные значения!

In [18]:
z = disallowmissing(y)
x,y,z

([1, 2, 3], Union{Missing, Int64}[1, 2, 3], [1, 2, 3])

В следующем примере мы покажем, что тип каждого столбца в `x` изначально является` Int64`. После использования `allowmissing!` Для принятия пропущенных значений в столбцах 1 и 3 типами этих столбцов становятся `Union {Int64, Missing}`.

In [19]:
x = DataFrame(Int, 2, 3)
println("Before: ", eltypes(x))
allowmissing!(x, 1) # make first column accept missings
allowmissing!(x, :x3) # make :x3 column accept missings
println("After: ", eltypes(x))

Before: DataType[Int64, Int64, Int64]
After: Type[Union{Missing, Int64}, Int64, Union{Missing, Int64}]


В следующем примере мы будем использовать `completecases`, чтобы найти все строки в DataFrame, которые содержат полные данные.

In [20]:
x = DataFrame(A=[1, missing, 3, 4], B=["A", "B", missing, "C"])

,A,B
,Int64⍰,String⍰
1,1,A
2,missing,B
3,3,missing
4,4,C


In [21]:
println("Complete cases:\n", completecases(x))

Complete cases:
Bool[true, false, false, true]


Мы можем использовать `dropmissing` или` dropmissing! `, Чтобы удалить строки с неполными данными из` DataFrame` и либо создать новый `DataFrame`, либо изменить исходный.

In [22]:
y = dropmissing(x)
dropmissing!(x)

┌ Warning: dropmissing will change eltype of cols to disallow missing by default. Use dropmissing(df, cols, disallowmissing=false) to allow for missing values.
│   caller = dropmissing(::DataFrame) at abstractdataframe.jl:643
└ @ DataFrames C:\Users\bogum\.julia\packages\DataFrames\lyCjP\src\abstractdataframe\abstractdataframe.jl:643
┌ Warning: dropmissing! will change eltype of cols to disallow missing by default. Use dropmissing!(df, cols, disallowmissing=false) to retain missing.
│   caller = dropmissing!(::DataFrame) at abstractdataframe.jl:733
└ @ DataFrames C:\Users\bogum\.julia\packages\DataFrames\lyCjP\src\abstractdataframe\abstractdataframe.jl:733


,A,B
,Int64⍰,String⍰
1,1,A
2,4,C


In [23]:
x

,A,B
,Int64⍰,String⍰
1,1,A
2,4,C


In [24]:
y

,A,B
,Int64⍰,String⍰
1,1,A
2,4,C


Когда мы вызываем метод `description` для объекта DataFrame с пропущенными значениями, столбцы по-прежнему допускают пропущенные значения. 

*В будущем планируется, что по умолчанию после этой операции в столбцах не будут пропущены значения.*

In [25]:
describe(x)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Int64,DataType
1,A,2.5,1,2.5,4,,0,Int64
2,B,,A,,C,2,0,String


Поскольку мы исключили пропущенные значения, мы можем смело использовать `disallowmissing!`, Чтобы столбцы больше не принимали пропущенные значения (мы можем видеть это как столбец `nmissing` пуст).

In [26]:
disallowmissing!(x)
describe(x)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,A,2.5,1,2.5,4,,,Int64
2,B,,A,,C,2,,String


В качестве альтернативы вы можете передать аргумент ключевого слова disallowmissing `dropmissing` и` dropmissing! `

In [27]:
x = DataFrame(A=[1, missing, 3, 4], B=["A", "B", missing, "C"])

,A,B
,Int64⍰,String⍰
1,1,A
2,missing,B
3,3,missing
4,4,C


In [28]:
dropmissing!(x, disallowmissing=true)

,A,B
,Int64,String
1,1,A
2,4,C


### Создание `missing`-ориентрированных функций 

Если у нас есть функция, которая не обрабатывает `missing` значения, мы можем обернуть ее с помощью функции «passmissing», чтобы в случае отсутствия какого-либо из ее позиционных аргументов мы получили взамен значение `missing`. В приведенном ниже примере мы изменим поведение функции string:

In [29]:
string(missing)

"missing"

In [30]:
string(missing, " ", missing)

"missing missing"

In [31]:
string(1,2,3)

"123"

In [32]:
lift_string = passmissing(string)

(::Missings.PassMissing{typeof(string)}) (generic function with 2 methods)

In [33]:
lift_string(missing)

missing

In [34]:
lift_string(missing, " ", missing)

missing

In [35]:
lift_string(1,2,3)

"123"